# HW 1

## Imports

In [ ]:
import nbtools
nbtools.setup_nb()

In [ ]:
import itertools
import sympy
import pandas
from scipy import integrate
import numpy
import plotly.express as px

from sympy.diffgeom import Manifold, Patch
from pynstein import coords, metric, curvature, geodesic
from pynstein.utilities import tensor_pow as tpow, full_simplify

## Exercises

### B1 - Polar Coords for $\mathbb{R}^2$

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

##### Cartesian

In [ ]:
x, y = sympy.symbols('x, y', nonnegative=False)
cs = coords.CoordSystem('cartesian', P, [x, y])
dx, dy = cs.base_oneforms()
ds2 = tpow(dx, 2) + tpow(dy, 2)
g_cart = metric.Metric(twoform=ds2)
g

##### Polar

In [ ]:
r, theta = sympy.symbols('r theta', nonnegative=True)
cs = coords.CoordSystem('polar', P, [r, theta])
dr, dtheta = cs.base_oneforms()
ds2 = tpow(dr, 2) + r ** 2 * tpow(dtheta, 2)
g = metric.Metric(twoform=ds2)
g

#### Compute curvature components

In [ ]:
crs, rmns, rcs = curvature.compute_components(g)

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rmns)

In [ ]:
curvature.display_components(rcs)

#### Compute Geodesics

In [ ]:
full_simplify(geodesic.geodesic_equation(0, sympy.symbols('lambda'), g))

In [ ]:
full_simplify(geodesic.geodesic_equation(1, sympy.symbols('lambda'), g))

##### Visualize Cartesian

In [ ]:
ls = numpy.arange(0, 2, 0.01)
init = (0.0, 0.0, 0.1, 0.1) 
df = geodesic.numerical_geodesic(g_cart, init, ls)

In [ ]:
ls = numpy.arange(0, 2, 0.001)
dfs = [geodesic.numerical_geodesic(g_cart, (0.0, 0.0, numpy.cos(theta_0), numpy.sin(theta_0)), ls)
               .assign(theta_0=theta_0) 
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, 0.2)]
df = pandas.concat(dfs, axis=0);

In [ ]:
fig = px.scatter(df, x="x", y='y', 
                 color='theta_0',
                 title="Sample Geodesic")

fig.show()

##### Visualize Polar

In [ ]:
ls = numpy.arange(0, 2, 0.01)
r_0 = 1.0
init = (r_0, 0.0, 0.0, numpy.pi/5) 
df = geodesic.numerical_geodesic(g, init, ls)

In [ ]:
ls = numpy.arange(0, 2, 0.001)
r_0 = 1.0 
dfs = [geodesic.numerical_geodesic(g, (r_0, theta_0, 0.0, numpy.pi/2), ls).assign(theta_0=theta_0) 
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, 0.2)]
df = pandas.concat(dfs, axis=0)

In [ ]:
df['theta'] = numpy.mod(df['theta'], 2*numpy.pi)
# df['lam'] = ls

In [ ]:
fig = px.scatter(df, x="theta", y='r', 
                 color='theta_0',
                 title="Sample Geodesic")

fig.show()

### B2 2-Sphere

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

theta, phi, a = sympy.symbols('theta phi a', nonnegative=True)
cs = coords.CoordSystem('spherical', P, [theta, phi])
dtheta, dphi = cs.base_oneforms()
ds2 = a**2 * (tpow(dtheta, 2) + sympy.sin(theta)**2 * tpow(dphi, 2))
g = metric.Metric(twoform=ds2)
g

#### Compute Components

In [ ]:
g.matrix

In [ ]:
g.inverse.matrix

In [ ]:
crs, rmn, rcs = curvature.compute_components(g)

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rmn)

In [ ]:
curvature.display_components(rcs)

In [ ]:
full_simplify(curvature.ricci_scalar(metric=g))

#### Geodesics

##### Geodesic Equations

In [ ]:
full_simplify(geodesic.geodesic_equation(0, sympy.symbols('lambda'), g))

In [ ]:
full_simplify(geodesic.geodesic_equation(1, sympy.symbols('lambda'), g))

##### Single Geodesic

In [ ]:
ls = numpy.arange(0, 8, 0.0001)
init = (0.01, 0.001, 3.14/4, 0.0)
st = 2
sp = 2
df = geodesic.numerical_geodesic(g, (numpy.pi/2, 0.0, numpy.pi/st, numpy.pi/sp), ls)
#        for theta_0 in numpy.arange(0.01, 3.14, 0.5)]

In [ ]:
df['theta'] = numpy.mod(df['theta'], 2*numpy.pi)
df['phi'] = numpy.mod(df['phi'], numpy.pi)
df = df.reset_index().rename(columns={'index': 'order'})
df['lam'] = ls

In [ ]:
fig = px.scatter(df, x="theta", y='phi', 
                 color='order',
                 title="B2 Geodesic")

fig.show()

##### Multiple Initial Conditions

In [ ]:
ls = numpy.arange(0, .04, 0.00001)
st = 2
sp = .01
dfs = [geodesic.numerical_geodesic(g, (theta_0, 0.0, numpy.pi/st, numpy.pi/sp), ls).assign(theta_0=theta_0) 
       for theta_0 in list(numpy.arange(numpy.pi/6, 5*numpy.pi/6, 0.1)) + 
       list(numpy.arange(numpy.pi/6 + numpy.pi, 5*numpy.pi/6 + numpy.pi, 0.1))]
#        for theta_0 in numpy.arange(0.01, 3.14, 0.5)]
df = pandas.concat(dfs, axis=0)

In [ ]:
df['theta'] = numpy.mod(df['theta'], 2*numpy.pi)
df = df.reset_index().rename(columns={'index': 'order'})

In [ ]:
min_phi = 0   
max_phi = 3.14  # Trim excess for plot widow

min_theta = 0
max_theta = 2*3.14

In [ ]:
lim_df = df[(df['phi'] >= min_phi) & (df['phi'] <= max_phi) &
            (df['theta'] >= min_theta) & (df['theta'] <= max_theta)]

In [ ]:
fig = px.scatter(lim_df, x="theta", y='phi', 
                 color='theta_0',
                 title="B2 Geodesics")

fig.show()

### B3 2-Sphere Sch

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

rho, phi, a = sympy.symbols('rho phi a', nonnegative=True)
cs = coords.CoordSystem('schw', P, [rho, phi])
drho, dphi = cs.base_oneforms()
ds2 = a**2 * ( (1 / (1 - rho**2)) * tpow(drho, 2) + rho ** 2 * tpow(dphi, 2))
g = metric.Metric(twoform=ds2)
g

#### Geodesics

In [ ]:
init = (numpy.sin(numpy.pi/4), 0.0, numpy.cos(numpy.pi/4), numpy.pi/4)
ts = numpy.arange(0, 2.1, 0.001)
df = geodesic.numerical_geodesic(g, init, ts)

In [ ]:
df['phi'] = numpy.mod(df['phi'], numpy.pi)
df = df.reset_index().rename(columns={'index': 'order'})
df['lam'] = ts

In [ ]:
fig = px.scatter(df, x="rho", y='phi', 
#                  color='init',
                     title="B3 Geodesic")

fig.show()

### B4 - Embedded Surface - General Case

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

x, y = sympy.symbols('x y', nonnegative=False)
_coords = [x, y]
cs = coords.CoordSystem('Cartesian', P, [x, y])
dx, dy = cs.base_oneforms()
_1forms = [dx, dy]

f = sympy.Function('f')(x, y)

ds2 = []
for i in range(2):
    for j in range(2):
        ds2.append(((1 if i == j else 0) + sympy.diff(f, _coords[i]) * sympy.diff(f, _coords[j])) 
                   * sympy.diffgeom.TensorProduct(_1forms[i],  _1forms[j]))
ds2 = sum(ds2)
g = metric.Metric(twoform=ds2)
g
g_b4 = g

#### Compute Components

In [ ]:
g.matrix

In [ ]:
g.inverse.matrix

In [ ]:
crs, rms, rcs = curvature.compute_components(g)

In [ ]:
# Simplify notation
subscript_notation = [
    (sympy.diff(sympy.diff(f, x), x), sympy.symbols('f_xx')),
    (sympy.diff(sympy.diff(f, y), y), sympy.symbols('f_yy')),
    (sympy.diff(sympy.diff(f, x), y), sympy.symbols('f_xy')),
    (sympy.diff(f, x), sympy.symbols('f_x')),
    (sympy.diff(f, y), sympy.symbols('f_y')),
]

In [ ]:
crs = [(c[0], c[1].subs(subscript_notation)) for c in crs]
rms = [(c[0], c[1].subs(subscript_notation)) for c in rms]
rcs = [(c[0], c[1].subs(subscript_notation)) for c in rcs]

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rms)

In [ ]:
curvature.display_components(rcs)

Clean Expr

In [ ]:
simplified = full_simplify(curvature.ricci_scalar(g)).subs(subscript_notation)
simplified

In [ ]:
# For posting / asking Eugenio
print(simplified._repr_latex_())

#### Fun Examples

Paraboloid

In [ ]:
a, b = sympy.symbols('a b')
subs = {f: x**2/a + y**2/b}

In [ ]:
full_simplify(full_simplify(curvature.ricci_scalar(g)).subs(subs))

### B5 - Embedded Surface - Sphere

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0**2 - x**2 - y**2)}
g = metric.Metric(twoform=g_b4.twoform.subs(subs))
g

#### Curvature

In [ ]:
full_simplify(curvature.ricci_scalar(g, simplify_intermediate=True))

#### Geodesic

In [ ]:
a_0_val = 1.0
g_num = metric.Metric(twoform=g.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
# init = (0.5, 0.5, -0.2, 0.2)
dfs = [geodesic.numerical_geodesic(g_num, (0.5, 0.5, 0.1*numpy.cos(theta_0), 0.1*numpy.sin(theta_0)), ls)
               .assign(theta_0=theta_0)
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, numpy.pi / 12)]
df = pandas.concat(dfs, axis=0)

In [ ]:
x_min = y_min = -1
x_max = y_max =  1

df = df[(df['x'] >= x_min) & (df['x'] <= x_max) &
        (df['y'] >= y_min) & (df['y'] <= y_max)]

In [ ]:
fig = px.scatter(df, x="x", y='y', 
                 color='theta_0',
                 title="B5 Geodesic")

fig.show()

### B6 - Embedded Surface - Hyperboloid

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0**2 - x**2 + y**2)}
g = metric.Metric(twoform=g_b4.twoform.subs(subs))
g

#### Curvature

In [ ]:
full_simplify(curvature.ricci_scalar(g, simplify_intermediate=True))

#### Geodesic

In [ ]:
a_0_val = 1.0
g_num = metric.Metric(twoform=g.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
# init = (0.5, 0.5, -0.2, 0.2)
dfs = [geodesic.numerical_geodesic(g_num, (0.5, 0.5, 0.1*numpy.cos(theta_0), 0.1*numpy.sin(theta_0)), ls)
               .assign(theta_0=theta_0)
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, numpy.pi / 12)]
df = pandas.concat(dfs, axis=0)

In [ ]:
x_min = y_min = -1
x_max = y_max =  1

df = df[(df['x'] >= x_min) & (df['x'] <= x_max) &
        (df['y'] >= y_min) & (df['y'] <= y_max)]

In [ ]:
fig = px.scatter(df, x="x", y='y', 
                 color='theta_0',
                 title="B6 Geodesic")

fig.show()

### B7 - Embedded Surface - Cylinder

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0**2 - y**2)}
g = metric.Metric(twoform=g_b4.twoform.subs(subs))
g

#### Curvature

In [ ]:
full_simplify(curvature.ricci_scalar(g))

#### Geodesic

In [ ]:
a_0_val = 1.0
g_num = metric.Metric(twoform=g.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
# init = (0.5, 0.5, -0.2, 0.2)
dfs = [geodesic.numerical_geodesic(g_num, (0.5, 0.5, 0.1*numpy.cos(theta_0), 0.1*numpy.sin(theta_0)), ls)
               .assign(theta_0=theta_0)
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, numpy.pi / 12)]
df = pandas.concat(dfs, axis=0)

In [ ]:
x_min = y_min = -1
x_max = y_max =  1

df = df[(df['x'] >= x_min) & (df['x'] <= x_max) &
        (df['y'] >= y_min) & (df['y'] <= y_max)]

In [ ]:
fig = px.scatter(df, x="x", y='y', 
                 color='theta_0',
                 title="B7 Geodesic")

fig.show()

### B8 - Embedded Surface - Cone

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(x**2 + y**2)}
g = metric.Metric(twoform=g_b4.twoform.subs(subs))
g

#### Curvature

In [ ]:
full_simplify(full_simplify(curvature.ricci_scalar(g)).subs(subs))

#### Geodesic

In [ ]:
a_0_val = 1.0
g_num = metric.Metric(twoform=g.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
# init = (0.5, 0.5, -0.2, 0.2)
dfs = [geodesic.numerical_geodesic(g_num, (0.5, 0.5, 0.1*numpy.cos(theta_0), 0.1*numpy.sin(theta_0)), ls)
               .assign(theta_0=theta_0)
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, numpy.pi / 12)]
df = pandas.concat(dfs, axis=0)

In [ ]:
x_min = y_min = -1
x_max = y_max =  1

df = df[(df['x'] >= x_min) & (df['x'] <= x_max) &
        (df['y'] >= y_min) & (df['y'] <= y_max)]

In [ ]:
fig = px.scatter(df, x="x", y='y', 
                 color='theta_0',
                 title="B7 Geodesic")

fig.show()